In [13]:
def ec_factorization(n, max_iterations=50, B_start=10):
    """
    parameters:
    - n: large integer to factorize
    - max_iterations: max number of elliptic curve iterations to try
    - B_start: starting value of B (upper bound for LCM)

    returns:
    - non-trivial factor of n if found; otherwise, None.
    """
    B = B_start
    while True:
        print(f"trying with B = {B}...")

        # calculate M = lcm(1, 2, ..., B)
        M = lcm(range(1, B + 1))
        print(f"Calculated M (lcm): {M}")

        # try different elliptic curves and points
        for iteration in range(max_iterations):
            a = randint(1, n - 1)
            b = randint(1, n - 1)

            # define elliptic curve modulo n
            try:
                E = EllipticCurve(Integers(n), [a, b])
            except ValueError:
                continue  # invalid curve (discriminant 0)

            print(f"using elliptic curve: {E}")

            # construct a point on the curve
            for _ in range(10):  # try 10 random x-values to find a valid point
                x = randint(0, n - 1)
                rhs = (x**3 + a*x + b) % n
                try:
                    y = mod(rhs, n).sqrt()  # try to compute y such that y^2 = rhs (mod n)
                    P = E([x, y])
                    print(f"Using point: {P}")
                    break
                except (ValueError, ArithmeticError):
                    continue  # no valid point found, try another x

            else:
                continue  # no valid point after 10 attempts, skip this curve

            # compute [M]P
            try:
                Q = M * P
                print(f"[{M}]P = {Q}")
            except ZeroDivisionError as err:
                # extract the problematic denominator from the error message
                err_message = str(err)
                problematic_value = int(err_message.split("inverse of ")[1].split(" ")[0])
                gcd_value = gcd(problematic_value, n)
                print(f"ZeroDivisionError encountered. GCD({problematic_value}, {n}) = {gcd_value}")
                if 1 < gcd_value < n:
                    return gcd_value

        # increment B and try again
        B += 10
        if B > 500:  # prevent runaway loops
            break

    return None

# test cases
test_numbers = [10403, 589, 8051, 9991]  # can replace with your test numbers

for num in test_numbers:
    print(f"factoring n = {num}...")
    factor = ec_factorization(num, max_iterations=30, B_start=10)
    if factor:
        print(f"found a non-trivial factor of {num}: {factor}")
    else:
        print(f"no factor found for {num}. try increasing B or max_iterations.")
    print("-" * 50)


Factoring n = 10403...
Trying with B = 10...
Calculated M (lcm): 2520
Using elliptic curve: Elliptic Curve defined by y^2 = x^3 + 9849*x + 4351 over Ring of integers modulo 10403
Using point: (3498 : 435 : 1)
[2520]P = (0 : 1 : 0)
Using elliptic curve: Elliptic Curve defined by y^2 = x^3 + 2586*x + 1326 over Ring of integers modulo 10403
Using point: (5104 : 1897 : 1)
ZeroDivisionError encountered. GCD(101, 10403) = 101
Found a non-trivial factor of 10403: 101
--------------------------------------------------
Factoring n = 589...
Trying with B = 10...
Calculated M (lcm): 2520
Using elliptic curve: Elliptic Curve defined by y^2 = x^3 + 207*x + 423 over Ring of integers modulo 589
Using point: (527 : 74 : 1)
[2520]P = (0 : 1 : 0)
Using elliptic curve: Elliptic Curve defined by y^2 = x^3 + 182*x + 156 over Ring of integers modulo 589
Using point: (79 : 141 : 1)
ZeroDivisionError encountered. GCD(31, 589) = 31
Found a non-trivial factor of 589: 31
-----------------------------------------